In [1]:
from bs4 import BeautifulSoup
from lib.import_lang_data import *
from lib.lang_featurizers import *
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier

In [2]:
# read in assignment's test files
X_assignment_test, y_assignment_test = read_tests()

# read corpus into lang_data, lang_results
languages = ['gcc', 'c', 'csharp', 'sbcl', 'clojure', 'ghc' 'java', 'javascript',
             'ocaml', 'perl', 'php', 'hack', 'py', 'python3', 'jruby', 'yarv', 'rb',
             'scala', 'racket']
lang_data, lang_results = read_polyglot(languages)
lang_info = pd.DataFrame(lang_results)
lang_info[0].value_counts()

gcc           58
scala         43
csharp        41
yarv          39
clojure       38
python3       36
ocaml         35
perl          34
sbcl          34
jruby         34
racket        29
php           29
hack          26
javascript    25
c              1
dtype: int64

In [3]:
lang_info = match_extensions(lang_info)
lang_results = list(lang_info[0])
lang_info[0].value_counts()

Ruby           73
C              59
PHP            55
Scala          43
C#             41
Clojure        38
Python         36
OCaml          35
Perl           34
Common Lisp    34
Scheme         29
JavaScript     25
dtype: int64

In [12]:
lang_featurizer = make_union(
    BagOfWordsFeaturizer(348),
    FunctionFeaturizer(presence_nil,
                       presence_nil_caps,
                       presence_null,
                       presence_none,
                       presence_start_double_semicolons,
                       presence_start_hashes,
                       presence_bar_hash,
                       percent_start_and_end_parenthesis,
                       presence_void,
                       presence_public,
                       presence_bool,
                       presence_module_line,
                       presence_extend_line,
                       presence_require_line,
                       presence_end,
                       presence_multiple_end,
                       presence_def_no_colon,
                       presence_at,
                       presence_double_at,
                       presence_defn,
                       percent_consecutive_closing_paren,
                       presence_from_import_line,
                       presence_import_line,
                       presence_print,
                       presence_dot_join,
                       presence_dot_format,
                       presence_dunder_name,
                       presence_dunder_init,
                       presence_def_colon,
                       
                       )
)

X_train, X_test, y_train, y_test = train_test_split(lang_data, lang_results)

pipe = make_pipeline(lang_featurizer, DecisionTreeClassifier())
# pipe = make_pipeline(lang_featurizer, MultinomialNB())
pipe.fit(X_train, y_train)
print('R^2 score: {}\n'.format(pipe.score(X_test, y_test)))
print(classification_report(pipe.predict(X_test), y_test))

R^2 score: 0.9444444444444444

             precision    recall  f1-score   support

          C       1.00      1.00      1.00        15
         C#       1.00      1.00      1.00         8
    Clojure       1.00      0.92      0.96        13
Common Lisp       0.92      1.00      0.96        11
 JavaScript       0.78      1.00      0.88         7
      OCaml       0.88      0.88      0.88         8
        PHP       1.00      0.73      0.85        15
       Perl       0.88      1.00      0.93         7
     Python       1.00      0.89      0.94         9
       Ruby       1.00      1.00      1.00        12
      Scala       0.94      1.00      0.97        16
     Scheme       0.83      1.00      0.91         5

avg / total       0.95      0.94      0.94       126



## Now to test with the assignment's tests

In [13]:
print('R^2 score: {}\n'.format(pipe.score(X_assignment_test, y_assignment_test)))
print(classification_report(pipe.predict(X_assignment_test), y_assignment_test))
print(confusion_matrix(pipe.predict(X_assignment_test), y_assignment_test))

R^2 score: 0.09375

             precision    recall  f1-score   support

    Clojure       0.25      0.20      0.22         5
    Haskell       0.00      0.00      0.00         0
       Java       0.00      0.00      0.00         0
 JavaScript       0.00      0.00      0.00         2
      OCaml       0.00      0.00      0.00         2
        PHP       0.33      0.09      0.14        11
     Python       0.25      0.25      0.25         4
       Ruby       0.00      0.00      0.00         3
      Scala       0.00      0.00      0.00         4
     Scheme       0.00      0.00      0.00         1
        TCL       0.00      0.00      0.00         0

avg / total       0.18      0.09      0.12        32

[[1 1 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 1 0 0]
 [1 0 2 1 1 1 0 1 1 2 1]
 [0 0 0 1 0 2 1 0 0 0 0]
 [0 0 0 0 0 0 3 0 0 0 0]
 [1 2 0 0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]]


/Users/ahartz1/TIY/programming-language-classifier/.direnv/python-3.4.3/lib/python3.4/site-packages/sklearn/metrics/classification.py:960: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
